In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append("drive")
import os
os.chdir('drive/My Drive/Colab Notebooks')

In [4]:
import pandas as pd
import numpy as np
import math
import itertools
from scipy.optimize import minimize

In [5]:
!pip install openpyxl
import openpyxl


In [6]:
#Import data
df = pd.read_excel('/content/drive/MyDrive/Data VN30(2).xlsx')
df['YearMonth'] = df['Date'].str[:4] + df['Date'].str[5:7]
df

,MCK,Date,<Close>,YearMonth
0,BCM,2019-05-01,22.38,201905
1,BCM,2019-06-01,21.63,201906
2,BCM,2019-07-01,23.21,201907
3,BCM,2019-08-01,27.11,201908
4,BCM,2019-09-01,32.82,201909
...,...,...,...,...
1615,VRE,2023-12-01,22.80,202312
1616,VRE,2024-01-01,23.30,202401
1617,VRE,2024-02-01,22.65,202402
1618,VRE,2024-03-01,26.65,202403


In [7]:
# Create an empty DataFrame
df0 = pd.DataFrame()

# Specify the file name for the new Excel file
excel_file1 = 'Portfolio.xlsx'

# Create ExcelWriter object
with pd.ExcelWriter(excel_file1) as writer:
    # Write the empty DataFrame to the Excel file
    df0.to_excel(writer, index=False)

print(f"A new blank Excel file '{excel_file1}' has been created.")

A new blank Excel file 'Portfolio.xlsx' has been created.


In [ ]:
def RpCombinationCal(name1, name2, name3, mean1, mean2, mean3, sd1, sd2, sd3, cov12, cov23, cov13):
    '''Calculate Rp and S.D'''
    combinationProportions = [(0.8, 0.1, 0.1), (0.7, 0.1, 0.2), (0.6, 0.3, 0.1),
                              (0.6, 0.2, 0.2), (0.6, 0.1, 0.3), (0.5, 0.4, 0.1),
                              (0.5, 0.3, 0.2), (0.5, 0.2, 0.3), (0.5, 0.1, 0.4),
                              (0.4, 0.5, 0.1), (0.4, 0.4, 0.2), (0.4, 0.3, 0.3),
                              (0.4, 0.2, 0.4), (0.4, 0.1, 0.5), (0.3, 0.6, 0.1),
                              (0.3, 0.5, 0.2), (0.3, 0.4, 0.3), (0.3, 0.3, 0.4),
                              (0.3, 0.2, 0.5), (0.3, 0.1, 0.6), (0.2, 0.7, 0.1),
                              (0.2, 0.6, 0.2), (0.2, 0.5, 0.3), (0.2, 0.4, 0.4),
                              (0.2, 0.3, 0.5), (0.2, 0.2, 0.6), (0.2, 0.1, 0.7),
                              (0.1, 0.8, 0.1), (0.1, 0.7, 0.2), (0.1, 0.6, 0.3),
                              (0.1, 0.5, 0.4), (0.1, 0.4, 0.5), (0.1, 0.3, 0.6),
                              (0.1, 0.2, 0.7), (0.1, 0.1, 0.8)]


    df = pd.DataFrame(combinationProportions, columns=[name1, name2, name3])
    df['Rp'] = df[name1] * mean1 + df[name2] * mean2 + df[name3] * mean3
    df['S.D'] = ((
            (df[name1] * sd1) ** 2
            + (df[name2] * sd2) ** 2
            + (df[name3] * sd3) ** 2
            + 2 * df[name1] * df[name2] * cov12
            + 2 * df[name2] * df[name3] * cov23
            + 2 * df[name1] * df[name3] * cov13
        ) ** 0.5)
    return df

In [ ]:
def maxSp(mean1, mean2, mean3, sd1, sd2, sd3, cov12, cov23, cov13):
    '''Calculate max Sp'''
    # Constants
    R1 = mean1  # Example values, replace with actual numbers
    R2 = mean2
    R3 = mean3
    rf = 0.002

    # Objective function
    def objective_function(weights):
        w1, w2, w3 = weights
        Rp = w1 * R1 + w2 * R2 + w3 * R3
        S = ((w1 * sd1) ** 2 + (w2 * sd2) ** 2 + (w3 * sd3) ** 2 +
            2 * w1 * w2 * cov12 + 2 * w2 * w3 * cov23 + 2 * w1 * w3 * cov13) ** 0.5
        return -((Rp-rf) / S)  # Maximize E(Rp)/S by minimizing its negative value

    # Constraint
    def sum_of_weights_constraint(weights):
        return 1 - np.sum(weights)

    # Define bounds and constraints
    bounds = [(0.05, 0.9)] * 3  # Weights must be between 0.05 and 0.9
    constraints = [{'type': 'eq', 'fun': sum_of_weights_constraint}]

    # Define step size
    step = 0.001

    # Define bounds with step size
    def bounds_with_step(step):
        return [(0.05 + step, 0.9 - step)] * 3  # Weights must be between 0.05 and 0.9 with step size

    # Optimization
    result = minimize(objective_function, [0.9, 0.05, 0.05], bounds=bounds_with_step(step), constraints=constraints)
    w1, w2, w3 = result.x
    Rp = w1 * R1 + w2 * R2 + w3 * R3
    S = ((w1 * sd1) ** 2 + (w2 * sd2) ** 2 + (w3 * sd3) ** 2 +
         2 * w1 * w2 * cov12 + 2 * w2 * w3 * cov23 + 2 * w1 * w3 * cov13) ** 0.5
    Sharpe_ratio = (Rp - rf) / S
    return w1, w2, w3, Rp, S, Sharpe_ratio

In [ ]:
def mergeDf(df, c):
    '''Create a unite dataframe'''

    name1, name2, name3 = c

    df1 = df[df['MCK']==name1].copy()
    df2 = df[df['MCK']==name2].copy()
    df3 = df[df['MCK']==name3].copy()

    df1.rename(columns={'<Close>':name1}, inplace=True)
    df2.rename(columns={'<Close>':name2}, inplace=True)
    df3.rename(columns={'<Close>':name3}, inplace=True)

    df1 = df1.set_index('YearMonth')
    df2 = df2.set_index('YearMonth')
    df3 = df3.set_index('YearMonth')

    dateSeries = df1['Date']
    dateDf = pd.DataFrame(dateSeries)
    newdf = pd.concat([df1, df2, df3], axis=1).drop(['MCK','Date'], axis=1)
    newdf.insert(loc = 0, column = dateDf.columns[0], value = dateDf.iloc[:,0])

    return newdf

In [ ]:
listCode = ['BCM',	'BID',	'BVH',	'CTG',	'FPT',	'GAS',	'GVR',	'HDB',	'HPG',	'MBB',	'MSN',	'MWG',	'PLX',	'POW',	'SAB',	'SSI',	'STB',	'TCB',	'TPB',	'VCB',	'VHM',	'VIB',	'VIC',	'VJC',	'VNM',	'VPB',	'VRE']

combinations = itertools.combinations(listCode, 3)
dataframes = []
i = 0
for c in combinations:

    #Set up variables
    newdf = mergeDf(df, c)
    name1, name2, name3 = c
    returnList1 = [np.nan] * len(newdf)
    returnList2 = [np.nan] * len(newdf)
    returnList3 = [np.nan] * len(newdf)

    #Calculate Return value
    for i in range(1, len(newdf)):
        returnList1[i] = ((newdf[name1].iloc[i] - newdf[name1].iloc[i-1])/ newdf[name1].iloc[i-1])
        returnList2[i] = ((newdf[name2].iloc[i] - newdf[name2].iloc[i-1])/ newdf[name2].iloc[i-1])
        returnList3[i] = ((newdf[name3].iloc[i] - newdf[name3].iloc[i-1])/ newdf[name3].iloc[i-1])

    #Add them to the df
    newdf['Return_' + name1] = returnList1
    newdf['Return_' + name2] = returnList2
    newdf['Return_' + name3] = returnList3

    #Mean
    mean1 = newdf['Return_'+ name1].mean()
    mean2 = newdf['Return_'+ name2].mean()
    mean3 = newdf['Return_'+ name3].mean()

    #Ri - mean
    subtractionMean1 = newdf['Return_'+ name1]-mean1
    subtractionMean2 = newdf['Return_'+ name2]-mean2
    subtractionMean3 = newdf['Return_'+ name3]-mean3
    newdf['(Ri - mean)_' + name1] = subtractionMean1
    newdf['(Ri - mean)_' + name2] = subtractionMean2
    newdf['(Ri - mean)_' + name3] = subtractionMean3

    #(Ra - mean) * (Rb - mean)
    newdf['{}*{}'.format(name1, name2)] = subtractionMean1*subtractionMean2
    newdf['{}*{}'.format(name1, name3)] = subtractionMean1*subtractionMean3
    newdf['{}*{}'.format(name2, name3)] = subtractionMean3*subtractionMean2
    newdf['(Ri - mean)^2_' + name1] = subtractionMean1**2
    newdf['(Ri - mean)^2_' + name2] = subtractionMean2**2
    newdf['(Ri - mean)^2_' + name3] = subtractionMean3**2

    #Descriptive summary
    n = newdf.shape[0]-1

    #Variance
    var1 = sum(newdf['(Ri - mean)^2_' + name1].dropna())/n
    var2 = sum(newdf['(Ri - mean)^2_' + name2].dropna())/n
    var3 = sum(newdf['(Ri - mean)^2_' + name3].dropna())/n

    #Standard deviation
    sd1, sd2, sd3 = math.sqrt(var1), math.sqrt(var2), math.sqrt(var3)

    #Cov
    cov12 = sum((subtractionMean1*subtractionMean2).dropna())/n
    cov23 = sum((subtractionMean2*subtractionMean3).dropna())/n
    cov13 = sum((subtractionMean1*subtractionMean3).dropna())/n

    #Cor
    cor12 = cov12/(sd1*sd2)
    cor23 = cov23/(sd2*sd3)
    cor13 = cov13/(sd1*sd3)

    #Names
    covDesc12 = 'Cov {}, {}'.format(name1, name2)
    covDesc23 = 'Cov {}, {}'.format(name2, name3)
    covDesc13 = 'Cov {}, {}'.format(name1, name3)
    corDesc12 = 'Cor {}, {}'.format(name1, name2)
    corDesc23 = 'Cor {}, {}'.format(name2, name3)
    corDesc13 = 'Cor {}, {}'.format(name1, name3)

    #Output
    output = pd.DataFrame({
        'Summary': ['Mean','Var','S.D','Cov','Cor.'],
        name1: [mean1, var1, sd1, cov12, cor12],
        name2: [mean2, var2, sd2, cov23, cor23],
        name3: [mean3, var3, sd3, cov13, cor13]
        })

    #Sum of columns
    sum12 = sum(newdf['{}*{}'.format(name1, name2)].dropna())
    sum23 = sum(newdf['{}*{}'.format(name2, name3)].dropna())
    sum13 = sum(newdf['{}*{}'.format(name1, name3)].dropna())

    sumRmean2_1 = sum(newdf['(Ri - mean)^2_' + name1].dropna())
    sumR2mean2_2 = sum(newdf['(Ri - mean)^2_' + name2].dropna())
    sumR2mean2_3 = sum(newdf['(Ri - mean)^2_' + name3].dropna())

    #Save all the sum into a df
    df_sum = pd.DataFrame(['Sum', sum12, sum23, sum13, sumRmean2_1, sumR2mean2_2, sumR2mean2_3])
    df_sum = df_sum.transpose()

    #Rp, SD
    Rp_SD = RpCombinationCal(name1, name2, name3, mean1, mean2, mean3, sd1, sd2, sd3, cov12, cov23, cov13)

    #MaxSp
    w1, w2, w3, E, S, MaxSp = maxSp(mean1, mean2, mean3, sd1, sd2, sd3, cov12, cov23, cov13)

    #Input
    inputDfPart2 = pd.DataFrame({
        'Summary': ['rf', name1, name2, name3, 'Cov({},{})'.format(name1, name2), 'Cov({},{})'.format(name2, name3), 'Cov({},{})'.format(name1, name3)],
        'E(r)': [0.002, mean1, mean2, mean3, cov12, cov23, cov13],
        'sigma': [0, sd1, sd2, sd3, np.nan, np.nan, np.nan]
    })

    #Output
    outputDfPart2 = pd.DataFrame({
        'Summary': ['W({})'.format(name1), 'W({})'.format(name2), 'W({})'.format(name3), 'Sum', 'E(rp)', 'sigmaP', 'Sharpe ratio'],
        'Value': [w1, w2, w3, round(w1 + w2 + w3, 0), E, S, MaxSp]
    })

    #Add to excel
    collected = {
        "Danh mục" : ['{}_{}_{}'.format(name1, name2, name3)],
        "w1" : [w1],
        "w2" : [w2],
        "w3" : [w3],
        "W" : [round(w1 + w2 + w3, 0)],
        "E(r)_1" : [mean1],
        "E(r)_2" : [mean2],
        "E(r)_3" : [mean3],
        "sd1" : [sd1],
        "sd2" : [sd2],
        "sd3" : [sd3],
        "cov12" : [cov12],
        "cov13" : [cov13],
        "cov23" : [cov23],
        "E(rp)" : [E],
        "sigmaP" : [S],
        "Sharpe ratio" : [MaxSp]

    }
    Portfolio = pd.DataFrame(collected)
    dataframes.append(Portfolio)

# Combine all DataFrames
all_data = pd.concat(dataframes, ignore_index=True)

# Write all DataFrames into a single sheet
all_data.to_excel(excel_file1, index=False)


print('Done')

